# **PDF QUERY USING LANGCHAIN :**

In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pdf-file/Pdf_8.pdf


In [7]:
!pip install PyPDF2

In [8]:
!pip install -U langchain-community

In [9]:
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

In [10]:
from langchain.embeddings.openai import OpenAIEmbeddings

In [11]:
import os
os.environ["GROQ_API_KEY"]="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"

## PDFREADER : 

In [48]:
pdfreader=PdfReader("/kaggle/input/llm-file/LLM_RAG_LangChain_Guide.pdf")

In [13]:
raw_text=""

In [49]:
for i,page in enumerate(pdfreader.pages):
    content=page.extract_text()
    if content :
        raw_text+=content

In [50]:
raw_text

'# NLP Preprocessing for RAG\nPreprocessing is essential to convert raw text into clean, structured data for embeddings and \nretrieval.\n## Common Preprocessing Steps\n1. **Tokenization**\n   - Split text into words or subwords.\n   - Libraries: NLTK, SpaCy, Hugging Face tokenizers.\n2. **Stopword Removal**\n   - Removes common words with little semantic value.\n   - Example: "the", "is", "and".\n3. **Stemming & Lemmatization**\n   - Stemming: Reduces words to base form (e.g., "running" → "run").\n   - Lemmatization: Converts to dictionary form using context.\n4. **Lowercasing & Cleaning**\n   - Remove punctuation, special characters, and normalize whitespace.\n## Advanced Preprocessing\n- **Named Entity Recognition (NER)**\n  - Tags proper nouns, organizations, locations, etc.\n- **POS Tagging**\n  - Identifies parts of speech to improve semantic understanding.\n- **Handling Tables and Code**\n  - Extract as separate chunks with metadata.Comprehensive Guide to LLMs, RAG, and LangChai

## Text Splitter : 

In [51]:
text_splitter=CharacterTextSplitter(
    separator="\n",
    chunk_size=800,
    chunk_overlap=200,
    length_function=len
)

In [52]:
texts=text_splitter.split_text(raw_text)

## Sentence Transformers Model : 

In [18]:
#!pip install sentence-transformers faiss-cpu langchain_groq

## Converting text into embeddings : 

In [53]:
from langchain.embeddings import HuggingFaceEmbeddings

In [54]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

## Document Search : 

In [21]:
#!pip install faiss-cpu

In [55]:
document_search = FAISS.from_texts(texts, embeddings)

In [56]:
print("✅ FAISS index created with", len(texts), "chunks")

✅ FAISS index created with 6 chunks


In [57]:
document_search

## Question Answering Chain : 

In [25]:
!pip uninstall -y langchain langchain-core

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Found existing installation: langchain 0.3.27
Uninstalling langchain-0.3.27:
  Successfully uninstalled langchain-0.3.27
Found existing installation: langchain-core 0.3.76
Uninstalling langchain-core-0.3.76:
  Successfully uninstalled langchain-core-0.3.76


In [29]:
!pip install langchain==0.3.27 langchain-core==0.3.76 --quiet


In [27]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [30]:
from langchain.chains import RetrievalQA

## ChatGroq Model :

In [31]:
from langchain_groq import ChatGroq

In [44]:
llm=ChatGroq(model="gemma2-9b-it")

## Retrieval QA :

In [33]:
from langchain.chains import RetrievalQA

In [58]:
qa_chain=RetrievalQA.from_chain_type(
    llm=llm,
    retriever=document_search.as_retriever(),
    chain_type="stuff"
)

## Query using chain : 

In [59]:
query = "Summarize the key points from the PDF."
answer=qa_chain.run(query)
print(answer)

This document does not contain a summary of the key points from the PDF. 



